In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Simulating dummy dataset
# Features: width, height, food_class, user_age, user_weight, activity_level
# Target: food_weight
np.random.seed(42)

num_samples = 1000
num_food_classes = 10

# Bounding box dimensions (width, height)
bounding_boxes = np.random.rand(num_samples, 2) * 100  # random width and height

# Food classes (encoded as integers)
food_classes = np.random.randint(0, num_food_classes, size=(num_samples, 1))

# User info (age, weight, activity level)
user_ages = np.random.randint(18, 60, size=(num_samples, 1))
user_weights = np.random.randint(50, 100, size=(num_samples, 1))
activity_levels = np.random.rand(num_samples, 1)  # random activity level between 0 and 1

# Food weight (target variable)
food_weights = np.random.rand(num_samples, 1) * 500  # random weight between 0 and 500 grams

# Combine all features into a single dataset
X = np.hstack((bounding_boxes, food_classes, user_ages, user_weights, activity_levels))
y = food_weights

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Check if GPU is available and move tensors to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)

# Define a neural network model
class FoodWeightEstimator(nn.Module):
    def __init__(self):
        super(FoodWeightEstimator, self).__init__()  
        self.fc1 = nn.Linear(X_tensor.shape[1], 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)  # Output: predicted food weight

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize model, loss function, and optimizer
model = FoodWeightEstimator().to(device)  # Move model to GPU
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 100000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")


Epoch [1000/100000], Loss: 18117.2617
Epoch [2000/100000], Loss: 15199.1250
Epoch [3000/100000], Loss: 12857.4189
Epoch [4000/100000], Loss: 11159.8281
Epoch [5000/100000], Loss: 10044.9805
Epoch [6000/100000], Loss: 9050.1680
Epoch [7000/100000], Loss: 8290.0537
Epoch [8000/100000], Loss: 7655.3472
Epoch [9000/100000], Loss: 7179.5078
Epoch [10000/100000], Loss: 7043.8496
Epoch [11000/100000], Loss: 6526.0923
Epoch [12000/100000], Loss: 6619.4258
Epoch [13000/100000], Loss: 6356.1904
Epoch [14000/100000], Loss: 5986.1343
Epoch [15000/100000], Loss: 6081.1660
Epoch [16000/100000], Loss: 5891.0122
Epoch [17000/100000], Loss: 5524.8672
Epoch [18000/100000], Loss: 5378.4712
Epoch [19000/100000], Loss: 5458.7480
Epoch [20000/100000], Loss: 5459.3184
Epoch [21000/100000], Loss: 5240.3452
Epoch [22000/100000], Loss: 5536.9346
Epoch [23000/100000], Loss: 4954.3779
Epoch [24000/100000], Loss: 4898.8184
Epoch [25000/100000], Loss: 4839.8989
Epoch [26000/100000], Loss: 4559.2432
Epoch [27000/100

In [7]:
import torch
import numpy as np

# Dummy nutritional database (simplified example)
nutritional_db = {
    0: {'name': 'rice', 'calories_per_100g': 130, 'protein_per_100g': 2.5, 'carbs_per_100g': 28, 'fats_per_100g': 0.3},
    1: {'name': 'chicken', 'calories_per_100g': 165, 'protein_per_100g': 31, 'carbs_per_100g': 0, 'fats_per_100g': 3.6},
    # Add more food items as needed...
}

# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to predict food weight using CUDA
def predict_food_weight(model, bounding_box, food_class, user_age, user_weight, activity_level):
    # Prepare input data
    input_data = np.array([[bounding_box[0], bounding_box[1], food_class, user_age, user_weight, activity_level]])
    input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)  # Move tensor to GPU
    
    # Ensure model is on the same device
    model = model.to(device)
    
    # Make prediction
    predicted_weight = model(input_tensor).item()  # Convert the prediction to a scalar value
    return predicted_weight

# Function to calculate nutritional information based on food weight
def calculate_nutrition(predicted_weight, food_class):
    food_info = nutritional_db[food_class]
    calories = (predicted_weight / 100) * food_info['calories_per_100g']
    protein = (predicted_weight / 100) * food_info['protein_per_100g']
    carbs = (predicted_weight / 100) * food_info['carbs_per_100g']
    fats = (predicted_weight / 100) * food_info['fats_per_100g']
    
    nutrition_info = {
        'Food': food_info['name'],
        'Predicted Weight (g)': predicted_weight,
        'Calories': calories,
        'Protein (g)': protein,
        'Carbs (g)': carbs,
        'Fats (g)': fats
    }
    return nutrition_info

# Function to give meal recommendation based on user profile
def recommend_meal(user_age, user_weight, activity_level):
    # Simplified recommendation logic based on user's profile
    if activity_level > 0.7:
        recommended_food = "High-protein meal: Grilled chicken with rice"
    else:
        recommended_food = "Balanced meal: Mixed vegetables with rice and tofu"
    
    recommendation = {
        'User Age': user_age,
        'User Weight': user_weight,
        'Activity Level': activity_level,
        'Meal Recommendation': recommended_food
    }
    return recommendation

# Example usage
# Assume model is already trained

# Input data
bounding_box = [50, 75]  # width, height of the food in pixels
food_class = 0  # class label for 'rice'
user_age = 25
user_weight = 70
activity_level = 0.9  # moderate activity level

# 1. Predict food weight
predicted_weight = predict_food_weight(model, bounding_box, food_class, user_age, user_weight, activity_level)
print(f"Predicted Food Weight: {predicted_weight:.2f} grams")

# 2. Calculate nutritional information
nutrition_info = calculate_nutrition(predicted_weight, food_class)
print("Nutritional Information:")
for key, value in nutrition_info.items():
    if isinstance(value, float):  # Check if the value is a float (number)
        print(f"{key}: {value:.2f}")  # Format numbers with 2 decimal places
    else:
        print(f"{key}: {value}")  # Print strings as they are

# 3. Provide meal recommendation
meal_recommendation = recommend_meal(user_age, user_weight, activity_level)
print("Meal Recommendation:")
for key, value in meal_recommendation.items():
    print(f"{key}: {value}")


Predicted Food Weight: -57.99 grams
Nutritional Information:
Food: rice
Predicted Weight (g): -57.99
Calories: -75.39
Protein (g): -1.45
Carbs (g): -16.24
Fats (g): -0.17
Meal Recommendation:
User Age: 25
User Weight: 70
Activity Level: 0.9
Meal Recommendation: High-protein meal: Grilled chicken with rice
